# **Código**

In [ ]:
# Instalar pygifsicle, para reducir tamaño de gifs
!pip install pygifsicle
!sudo apt-get install gifsicle

In [127]:
from PIL import Image
import time
import os
import pygifsicle
import shutil # Mover archivos generados, al final

In [201]:
# @Oscar-gg

# Class to perform image reduction / compressing to reduce file size,
# taking into account image type and options
# Supports: .png, .jpg

class ImageReduction:

  def __init__(self, threshold_kb = 500, min_quality=60,
               extensions=[".png", ".jpg",".jpeg", ".gif"],
               max_img_width_px=1000, max_img_height_px=1000, resize=False, reduce_all_valid_files=False,
               output_reduce_prefix="IR_", output_resize_prefix = "R_", best_prefix="B_", max_resize=0.6,
               colors_=32, scale_ratio='0.5', lossiness_factor='80'):

    self.threshold_kb = threshold_kb # Reducir solo imágenes que tengan peso mayor

    # La menor calidad a intentar (se usan calidades mayores primero), para jpg
    self.min_quality = min_quality

    # Solo hay implementación para .png y .jpg. Si se agregan más solo apareceran
    # en print_files_above_threshold(). Quitar .png o .jpg para ignorar archivos
    self.extensions = extensions

    # Si es verdadero, al reducir una imagen se hace resize().
    self.resize = resize

    # Si es verdadero, va a intentar reducir todas las imágenes,
    # incluso las que estén por debajo del threshold.
    # Muy poco recomendable ponerlo en true;
    self.reduce_all_valid_files = reduce_all_valid_files

    # Limites de imagen. Se usan cuando resize esta activo
    self.max_img_width_px = max_img_width_px
    self.max_img_height_px = max_img_height_px

    self.output_reduce_prefix = output_reduce_prefix
    self.output_resize_prefix = output_resize_prefix
    self.best_prefix = best_prefix

    # Evitar sobreescribir imágenes originales
    if len(self.output_reduce_prefix) == 0:
      self.output_reduce_prefix = "IR_"

    if len(self.output_resize_prefix) == 0:
      self.output_resize_prefix = "R_"

    if len(self.best_prefix) == 0:
      self.best_prefix = "B_"

    # Solo se usa con resize_directory
    self.max_resize = max_resize

    # Opciones para gifs
    self.colors_= colors_
    self.scale_ratio = scale_ratio
    self.lossiness_factor = lossiness_factor


  def reduce_directory(self, path_to_directory):
    files_to_reduce = []

    if self.reduce_all_valid_files:
      files_to_reduce = self.all_valid_files(path_to_directory)
    else:
      files_to_reduce = self.files_above_threshold(path_to_directory)

    for size, file_path in files_to_reduce:
      self.reduce_image(file_path)


  def resize_limits_directory(self, path_to_directory, all=False):
    """Only resizes valid images to max width or max height specified."""
    files_to_resize=[]
    if all:
      files_to_resize = self.all_valid_files(path_to_directory)
    else:
      files_to_resize = self.files_above_threshold(path_to_directory)

    for size, file_path in files_to_resize:
      image = Image.open(file_path)
      if self.over_dimensions(image):
        output_path = self.insert_prefix(self.output_resize_prefix, file_path)

        if not os.path.exists(output_path):
            resized = self.reduce_dimensions(image)
            resized.save(output_path, optimize=True)
            self.display_result(output_path, file_path)
        else:
            self.display_result("exists", output_path)

      else:
        print(f"Info: image is within specified limit dimensions: {file_path}")


  def resize_directory(self, path_to_directory):
    """Attempts to resize all valid files that are above threshold size."""
    files_to_resize = []
    files_to_resize = self.files_above_threshold(path_to_directory)

    for size, file_path in files_to_resize:
      self.resize_image(file_path)


  def resize_image(self, path_to_file):
    """Resize several times until being below threshold or reaching the limit."""
    if not self.valid_file(path_to_file):
      print(f"Warning: file not valid {path_to_file}.")
      return

    output, input = "", ""

    if self.file_extension(path_to_file) in [".jpg", ".jpeg", ".png"]:
      output, input= self.resize_png_jpg(path_to_file)

    if self.file_extension(path_to_file) in [".gif"]:
      output, input = self.compress_gif(path_to_file)

    self.display_result(output, input)

  def resize_png_jpg(self, path_to_file):
    if self.file_extension(path_to_file) not in [".jpg", ".jpeg", ".png"]:
      print(f"Warning: invalid path for resize_png_jpg, {path_to_file}. Method will skip")
      return [path_to_file, path_to_file]

    image = Image.open(path_to_file)
    output_path = self.insert_prefix(self.output_resize_prefix, path_to_file)

    # Avoid processing an image again.
    if os.path.exists(output_path):
        return ["exists", output_path]

    initial_size = 1
    current_size = initial_size
    resize_interval = (initial_size - self.max_resize) / 5 # Check 5 intervals

    image.save(output_path, optimize=True) # Save original image

    is_png = self.file_extension(path_to_file) == ".png"

    while self.get_file_size_kb(output_path) > self.threshold_kb and current_size >= self.max_resize and resize_interval > 0:
      start_total = time.time()
      current_size -= resize_interval
      resize_iteration = image.copy()
      resize_iteration = self.direct_resize(resize_iteration, current_size)

      if is_png:
        resize_iteration.save(output_path, optimize=True)
      else:
        resize_iteration.save(output_path, quality=80, optimize=True)

      end_time = time.time()

      difference_time = end_time - start_total
      if (difference_time) > 2:
        print(f"Warning: resize iteration took too long ({difference_time} s) for {path_to_file}. Trying only the last iteration.")
        resize_iteration = image.copy()
        resize_iteration = self.direct_resize(resize_iteration, self.max_resize)
        if is_png:
          resize_iteration.save(output_path, optimize=True)
        else:
          resize_iteration.save(output_path, quality=80, optimize=True)
        break

    return [output_path, path_to_file]


  def reduce_image(self, path_to_file):
    if not self.valid_file(path_to_file):
      print(f"Warning: file not valid {path_to_file}.")
      return

    output, input = "", ""

    if self.file_extension(path_to_file) in [".jpg", ".jpeg"]:
      output, input= self.reduce_jpg(path_to_file)
    elif self.file_extension(path_to_file) in [".png"]:
      output, input = self.reduce_png(path_to_file)
    elif self.file_extension(path_to_file) in [".gif"]:
      output, input = self.compress_gif(path_to_file)

    self.display_result(output, input)


  def display_result(self, output, input):
    if output == "" or input == "":
      print(f"Info: file not processed {input}.")
    elif output == "exists":
      print(f"Info: skipping, file already exists {input}.")
    else:
      out_size = self.get_file_size_kb(output)
      in_size = self.get_file_size_kb(input)

      percentage = (out_size / in_size) * 100
      print(f"[{round(100 - percentage, 2)}% reduction | {in_size} kb -> {out_size} kb: {output}]")

      self.was_resized(output, input)

      if percentage == 100:
        print(f"Warning: file size unchanged for {output}.")
      if percentage > 100:
        print(f"WARNING: file size INCREASED for {output}.")
      if percentage < 100 and out_size > self.threshold_kb:
        print(f"Warning: file size reduced but image remains above theshold for {output}.")

  def was_resized(self, output_path, input_file):
    if os.path.basename(output_path).startswith(self.output_resize_prefix):
        input_image = Image.open(input_file)
        output_image = Image.open(output_path)
        print(f"Output: {output_image.width} w {output_image.height} h, Input: {input_image.width} w {input_image.height} h")


  def reduce_png(self, path_to_file):
    if self.file_extension(path_to_file) not in [".png"]:
      print(f"Warning: invalid path for reduce_png, {path_to_file}. Method will skip")
      return [path_to_file, path_to_file]

    image = Image.open(path_to_file)

    output_path = self.insert_prefix(self.output_reduce_prefix, path_to_file)

    if os.path.exists(output_path):
        return ["exists", output_path]

    reduced_png = image.copy()

    if self.resize:
      reduced_png = self.reduce_dimensions(reduced_png)

    reduced_png = reduced_png.quantize(colors=256) # https://pillow.readthedocs.io/en/stable/reference/Image.html#PIL.Image.Image.quantize

    # PNG saving options: https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#png
    reduced_png.save(output_path, optimize=True)
    return [output_path, path_to_file]


  def reduce_jpg(self, path_to_file):
    if self.file_extension(path_to_file) not in [".jpg", ".jpeg"]:
      print(f"Warning: invalid path for reduce_jpg, {path_to_file}. Method will skip")
      return [path_to_file, path_to_file]

    image = Image.open(path_to_file)

    reduced_jpg = image.copy()

    if self.resize:
      reduced_jpg = self.reduce_dimensions(reduced_jpg)

    # JPEG saving options: https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#jpeg-saving

    output_path = self.insert_prefix(self.output_reduce_prefix, path_to_file)

    if os.path.exists(output_path):
        return ["exists", output_path]

    reduced_jpg.save(output_path)

    initial_quality = 90
    current_quality = initial_quality
    quality_interval = int((initial_quality - self.min_quality) / 5) # Check 5 intervals

    while self.get_file_size_kb(output_path) > self.threshold_kb and current_quality >= self.min_quality and quality_interval > 0:
      reduce_iteration = reduced_jpg.copy()

      reduce_iteration.save(output_path, optimize=True, quality=current_quality)
      current_quality -= quality_interval

    return [output_path, path_to_file]


  def reduce_dimensions(self, image):
    """Reduces image to max dimensions. Maintains aspect ratio: considers only max_img_width_px or max_img_height_px."""
    reduced_dim = image.copy()

    try:
      if reduced_dim.width > self.max_img_width_px and self.max_img_width_px > 0:
        width_reduction = self.max_img_width_px / image.width
        return self.direct_resize(reduced_dim, width_reduction)

      if reduced_dim.height > self.max_img_height_px and self.max_img_height_px > 0:
        height_reduction = self.max_img_height_px / image.height
        return self.direct_resize(reduced_dim, height_reduction)

    except:
      print("Error: Some error happened during resizing.")

    print(f"Warning: reduce_dimensions didn't modify the image. original: width({image.width}), height ({image.height}). Max dimensions: width({self.max_img_width_px}), height ({self.max_img_height_px})")
    return reduced_dim


  def compress_gif(self, path_to_file):
    scale_factor = f'{self.scale_ratio}x{self.scale_ratio}'

    output_path = self.insert_prefix(self.output_reduce_prefix, path_to_file)

    options_string = ["--verbose", f"--lossy={self.lossiness_factor}", f"--scale={scale_factor}"]
    pygifsicle.gifsicle(
      sources=[path_to_file], # or a single_file.gif
      destination=output_path, # or just omit it and will use the first source provided.
      optimize=True, # Whetever to add the optimize flag of not
      colors=self.colors_, # Number of colors to use
      options=options_string # Options to use.
    )

    return [output_path, path_to_file]

  def over_dimensions(self, image):
    return image.width > self.max_img_width_px or image.height > self.max_img_height_px


  def direct_resize(self, image, ratio):
    new_width = int(image.width * ratio)
    new_height = int(image.height * ratio)
    resized_image = image.resize((new_width, new_height))
    # See resampling filters: https://pillow.readthedocs.io/en/stable/handbook/concepts.html#PIL.Image.Resampling.LANCZOS
    return resized_image


  def file_extension(self, path_to_file):
    _, ext = os.path.splitext(path_to_file)
    return ext


  def valid_file(self, path_to_file):
    file_ext = self.file_extension(path_to_file)
    return file_ext in self.extensions


  def files_above_threshold(self, path_to_directory):
    return self.all_valid_files(path_to_directory, self.threshold_kb)


  def all_valid_files(self, path_to_directory, threshold_kb_=0):
    valid_files = []
    for root, _, files in os.walk(path_to_directory):
      for file in files:
        file_path = os.path.join(root, file)

        if not self.valid_file(file_path):
          continue

        size = self.get_file_size_kb(file_path)
        if (size >= threshold_kb_):
          valid_files.append((size, file_path))

    list.sort(valid_files, reverse=True)
    return valid_files


  def print_files_above_threshold(self, path_to_directory):
    files_over_threshold = self.files_above_threshold(path_to_directory)
    for file in files_over_threshold:
      print(f"{file[0]} KB: {file[1]}")


  def generated_files(self, path_to_directory):
    generated_files = []
    for root, _, files in os.walk(path_to_directory):
      for file in files:
        file_path = os.path.join(root, file)

        if not self.valid_file(file_path):
          continue

        if self.file_was_generated(file):
          size = self.get_file_size_kb(file_path)
          generated_files.append((size, file_path))

    list.sort(generated_files, reverse=True)
    return generated_files


  def file_was_generated(self, path_to_file):
    file_basename = os.path.basename(path_to_file)
    return file_basename.startswith(self.output_reduce_prefix) or file_basename.startswith(self.output_resize_prefix) or file_basename.startswith(self.best_prefix)


  def insert_prefix(self, prefix, path_to_file):
    directory = os.path.dirname(path_to_file)
    filename = prefix + os.path.basename(path_to_file)
    return os.path.join(directory, filename)


  def print_generated_files(self, path_to_directory, only_above_threshold=False):
    below_threshold = 0
    generated_files = self.generated_files(path_to_directory)

    for file in generated_files:
      if (file[0] < self.threshold_kb):
        below_threshold += 1

    if len(generated_files) == 0:
      print("No generated files found.")
      return

    percentage = int((below_threshold / len(generated_files)) * 100)
    print(f'{percentage}% of generated files are below threshold')

    for file in generated_files:
      if only_above_threshold and file[0] < self.threshold_kb:
         break
      print(f"[{file[0]} KB]: {file[1]}")


  def get_file_size_kb(self, file_path):
    size = os.path.getsize(file_path)
    return size // 1024  # bytes a kb

  def move_generated(self, directory_path, output_directory, only_basename=False):
    generated_files = self.generated_files(directory_path)

    destination_directory = os.path.dirname(output_directory)
    os.makedirs(destination_directory, exist_ok=True)

    for file in generated_files:
      destination_path = ""

      if self.is_contained(output_directory, file[1]):
        print(f"Info: {file[1]} is contained in {output_directory}. Skipping.")
        continue

      if only_basename:
        destination_path = os.path.join(output_directory, os.path.basename(file[1]))
      else:
        destination_path = os.path.join(output_directory, file[1])
        destination_directory = os.path.dirname(destination_path)

        os.makedirs(destination_directory, exist_ok=True)

      shutil.move(file[1], destination_path)

      print(f"Info: moved {file} to {destination_path}.")

  def is_contained(self, directory_path, file_path):
    directory_path = os.path.abspath(directory_path)
    file_path = os.path.abspath(file_path)

    return file_path.startswith(directory_path + os.sep)

  def check_remaining_files(self, directory_path):
    generated_files, original_files = self.generated_and_original(directory_path)
    original_set = set()

    # Save original basenames of files that are above threshold.
    for file in original_files:
      if file[0] > self.threshold_kb:
        original_set.add(os.path.basename(file[1]))

    for file in generated_files:
      if file[0] < self.threshold_kb:
        original_name = self.trim_prefix(file[1])

        if original_name in original_set:
          original_set.remove(original_name)


    if len(original_set) == 0:
      print(f"There is at least one version of each file that is below {self.threshold_kb} kb.")
    else:
      print(f"There are {len(original_set)} files that are above {self.threshold_kb} kb and have no generated version.")
      for file in original_set:
        print(f"  {file}")

  def generated_and_original(self, directory_path):
    all_files = self.all_valid_files(directory_path)
    generated_files = []
    original_files = []
    for file in all_files:
      if self.file_was_generated(file[1]):
        generated_files.append(file)
      else:
        original_files.append(file)

    return generated_files, original_files


  def trim_prefix(self, path_to_file):
    basename = os.path.basename(path_to_file)
    if basename.startswith(self.output_reduce_prefix):
      return self.trim_prefix(basename[len(self.output_reduce_prefix):])
    elif basename.startswith(self.output_resize_prefix):
      return self.trim_prefix(basename[len(self.output_resize_prefix):])
    elif basename.startswith(self.best_prefix):
      return self.trim_prefix(basename[len(self.best_prefix):])

    return basename


  def modify_gif_options(self, colors_: int, scale_ratio: str, lossiness_factor: str):
    self.colors_= colors_
    self.scale_ratio = scale_ratio
    self.lossiness_factor = lossiness_factor

  # Only method that can remove files. Maintains original files.
  def save_only_smallest_modified_files(self, directory_path):
    for root, _, files in os.walk(directory_path):
      for file in files:

        file_path = os.path.join(root, file)

        if not self.valid_file(file_path) or not self.file_was_generated(file_path):
          continue

        original_path = os.path.join(root, self.trim_prefix(file_path))

        best_path = self.insert_prefix(self.best_prefix, original_path)

        if not os.path.exists(best_path):
          shutil.move(file_path, best_path)
          continue

        if self.get_file_size_kb(file_path) < self.get_file_size_kb(best_path):
          shutil.move(file_path, best_path)
        else:
          os.remove(file_path)

# **Ejemplos de uso**

## **Setup en colab**

In [ ]:
# Subir archivos desde drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar archivos a libreta
!unzip /content/drive/MyDrive/cva/IlustracionesNuevas.zip

Archive:  /content/drive/MyDrive/cva/IlustracionesNuevas.zip
   creating: Ilustraciones nuevas/Barra de navegaciвn/
   creating: Ilustraciones nuevas/Barra de navegaciвn/Contаctanos/
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Contаctanos/BarraNavegecion_Contactanos.png  
   creating: Ilustraciones nuevas/Barra de navegaciвn/Creditos/
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Creditos/BarraNavegecion_Creditos.png  
   creating: Ilustraciones nuevas/Barra de navegaciвn/Fin de curso/
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Fin de curso/Fin De Curso_transparente-01.png  
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Fin de curso/Fin De Curso-01.png  
   creating: Ilustraciones nuevas/Barra de navegaciвn/Introducciвn/
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Introducciвn/CVA_migrantes_finales_1-01.jpg  
  inflating: Ilustraciones nuevas/Barra de navegaciвn/Introducciвn/Migrantes_bienvenida-01-01.jpg  
  inflating: Ilustraciones nueva

## **Uso de script**

### **Setup**

In [202]:
# Terminos usados:
# -resize: cambiar dimensiones de imagen. En el script siempre se mantiene el radio original de la imagen.
# -reduce: bajar peso de archivo sin modificar dimensiones.
#    - Se puede hacer reduciendo: los colores, aumentando compresion, o bajando calidad

# Instanciar clase
threshold_kb = 500 # Reducir imagenes que sean más grandes
min_quality = 60 # Rango [0, 100]. Usado para jpg.
extensions = [".png", ".jpg", ".jpeg" ,".gif"] # Extensiones de archivos a procesar
max_img_width_px = 1000 # Reducir imagenes que sean más anchas
max_img_height_px = 1000 # Reducir imagenes que sean más altas
resize = False # Hacer resize a dimensiones máximas al llamar reduce_jpg o reduce_png

reduce_all_valid_files = False # Si es verdadero, va a intentar reducir todas las imágenes, incluso las que estén por debajo del threshold. Poco recomendable ponerlo en true;
output_prefix = "IR_" # Prefijo de imágenes reducidas
output_resize_prefix = "R_" # prefijo de imágenes 'resized'
best_prefix = "B_" # prefijo para cuando se guarda solo la mejor imagen / gif.

max_resize = 0.6 # Rango: (0.0, 1.0) La máxima proporción a reducir el tamaño

# Opciones para gifs
colors_= 32 # Cantidad de colores en imagen. Tiene que ser potencia de 2
scale_ratio = '0.5' # (0.0, 1.0] resize en la imagen
lossiness_factor = '80' # [0, 100] Entre más alto menor calidad y peso de archivo

imagenes_proyecto = ImageReduction(threshold_kb, min_quality,extensions,
                                   max_img_width_px, max_img_height_px, resize,
                                   reduce_all_valid_files, output_prefix,
                                   output_resize_prefix, best_prefix, max_resize, colors_,
                                   scale_ratio, lossiness_factor)

In [197]:
# Conseguir path del directorio de imagenes

#os.chdir("IlustracionesNuevas")
directory_path = "IlustracionesNuevas"
print(os.listdir(directory_path))

['Mвdulo 5', 'Modulo1', 'Mвdulo 3', 'Mвdulo 4', 'Mвdulo 2', 'Barra de navegaciвn']


### **Ver resultados**

In [209]:
# Conseguir imágenes que son más pesadas del límite establecido
imagenes_proyecto.print_files_above_threshold(directory_path)

6297 KB: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/2-Pedro.gif
6154 KB: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/2-Juanita.gif
3375 KB: IlustracionesNuevas/Mвdulo 5/Extras/UsodeServicios.gif
2547 KB: IlustracionesNuevas/Mвdulo 2/2.1 Diferencias entre la concepciвn de salud/esquemaUNIDAD2.gif
2455 KB: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/Migrantes_bienvenida-01-01.jpg
2398 KB: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/2-Unidad2_BanderaMexico.gif
1830 KB: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/2-Unidad2_BanderaEEUU-copy.gif
1662 KB: IlustracionesNuevas/Barra de navegaciвn/Fin de curso/Fin De Curso-01.png
1609 KB: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/CVA_migrantes_finales_1-01.jpg
1472 KB: IlustracionesNuevas/Barra de navegaciвn/Fin de curso/Fin De Curso_transparente-01.png
1264 KB: IlustracionesNuevas/Modulo1/1.3RemediosCaseros/Unidad1GIF_RemediosCaseros.gif
1079 KB: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/2.2-Esquem

In [208]:
# Conseguir imágenes generadas
imagenes_proyecto.print_generated_files(directory_path, only_above_threshold=False)


90% of generated files are below threshold
[996 KB]: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/B_2-Juanita.gif
[928 KB]: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/B_2-Pedro.gif
[457 KB]: IlustracionesNuevas/Barra de navegaciвn/Fin de curso/B_Fin De Curso-01.png
[411 KB]: IlustracionesNuevas/Mвdulo 2/2.1 Diferencias entre la concepciвn de salud/B_esquemaUNIDAD2.gif
[409 KB]: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/B_2.2-EsquemaSituacionMX.gif
[406 KB]: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/B_CVA_migrantes_finales_1-01.jpg
[389 KB]: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/B_Migrantes_bienvenida-01-01.jpg
[377 KB]: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/B_2-Unidad2_BanderaMexico.gif
[368 KB]: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/B_2-Unidad2_BanderaEEUU-copy.gif
[368 KB]: IlustracionesNuevas/Modulo1/1.3RemediosCaseros/B_Unidad1GIF_RemediosCaseros.gif
[347 KB]: IlustracionesNuevas/Barra de navegaciвn/Creditos/B_BarraNav

In [207]:
# Checar cuáles imagenes no tienen una version menor al threshold en peso
imagenes_proyecto.check_remaining_files(directory_path)

There are 2 files that are above 500 kb and have no generated version.
  2-Juanita.gif
  2-Pedro.gif


### **Optimizar imagenes**

In [144]:
# Reducir imágenes que sobrepasan el threshold
imagenes_proyecto.reduce_directory(directory_path)

[63.36% reduction | 6297 kb -> 2307 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_2-Pedro.gif]
[60.19% reduction | 6154 kb -> 2450 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_2-Juanita.gif]
[81.19% reduction | 3375 kb -> 635 kb: IlustracionesNuevas/Mвdulo 5/Extras/IR_UsodeServicios.gif]
[57.44% reduction | 2547 kb -> 1084 kb: IlustracionesNuevas/Mвdulo 2/2.1 Diferencias entre la concepciвn de salud/IR_esquemaUNIDAD2.gif]
[69.86% reduction | 2455 kb -> 740 kb: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/IR_Migrantes_bienvenida-01-01.jpg]
[84.28% reduction | 2398 kb -> 377 kb: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/IR_2-Unidad2_BanderaMexico.gif]
[79.89% reduction | 1830 kb -> 368 kb: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/IR_2-Unidad2_BanderaEEUU-copy.gif]
[69.31% reduction | 1662 kb -> 510 kb: IlustracionesNuevas/Barra de navegaciвn/Fin de curso/IR_Fin De Curso-01.png]
[70.54% reduction | 1609 kb -> 474 kb: IlustracionesNuevas/Barra de

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (104183334 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[67.73% reduction | 1032 kb -> 333 kb: IlustracionesNuevas/Barra de navegaciвn/Metodologбa/IR_BarraNavegecion_Metodologia.png]


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (100020001 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[68.9% reduction | 939 kb -> 292 kb: IlustracionesNuevas/Barra de navegaciвn/Polбticas/IR_BarraNavegacion_Politicas.png]
[61.95% reduction | 912 kb -> 347 kb: IlustracionesNuevas/Barra de navegaciвn/Creditos/IR_BarraNavegecion_Creditos.png]
[66.9% reduction | 725 kb -> 240 kb: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/IR_2.3-EsquemaSituacionUSA.gif]
[62.0% reduction | 600 kb -> 228 kb: IlustracionesNuevas/Mвdulo 4/4B. Servicios pгblicos para familias de bajos recursos/4B1. Medicaid/4B. Requisitos para aplicar a Medicaid/IR_u4_requisitosParaAplicar-01.png]
[57.0% reduction | 600 kb -> 258 kb: IlustracionesNuevas/Barra de navegaciвn/Objetivos/IR_Objetivos.png]
[57.0% reduction | 600 kb -> 258 kb: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/IR_Objetivos.png]
[67.08% reduction | 571 kb -> 188 kb: IlustracionesNuevas/Mвdulo 4/4.0 Bienvenida/IR_Unidad4_Introduccion.png]
[56.21% reduction | 523 kb -> 229 kb: IlustracionesNuevas/Mвdulo 4/4B. Servicios pгblicos para familias de b

In [146]:
# Hacer resize a imágenes que sobrepasan el threshold
imagenes_proyecto.resize_directory(directory_path)

[63.36% reduction | 6297 kb -> 2307 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_2-Pedro.gif]
[60.19% reduction | 6154 kb -> 2450 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_2-Juanita.gif]
[81.19% reduction | 3375 kb -> 635 kb: IlustracionesNuevas/Mвdulo 5/Extras/IR_UsodeServicios.gif]
[57.44% reduction | 2547 kb -> 1084 kb: IlustracionesNuevas/Mвdulo 2/2.1 Diferencias entre la concepciвn de salud/IR_esquemaUNIDAD2.gif]
[84.15% reduction | 2455 kb -> 389 kb: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/R_Migrantes_bienvenida-01-01.jpg]
Output: 12500 w 2500 h, Input: 20834 w 4167 h
[59.35% reduction | 2450 kb -> 996 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_IR_2-Juanita.gif]
[84.28% reduction | 2398 kb -> 377 kb: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/IR_2-Unidad2_BanderaMexico.gif]
[59.77% reduction | 2307 kb -> 928 kb: IlustracionesNuevas/Mвdulo 3/3.2 Tipos de seguros/IR_IR_2-Pedro.gif]
[79.89% reduction | 1830 kb -> 368 kb: Ilu

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (104183334 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[22.77% reduction | 1032 kb -> 797 kb: IlustracionesNuevas/Barra de navegaciвn/Metodologбa/R_BarraNavegecion_Metodologia.png]
Output: 5000 w 7500 h, Input: 8334 w 12501 h


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (100020001 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[20.77% reduction | 939 kb -> 744 kb: IlustracionesNuevas/Barra de navegaciвn/Polбticas/R_BarraNavegacion_Politicas.png]
Output: 6000 w 6000 h, Input: 10001 w 10001 h
[1.1% reduction | 912 kb -> 902 kb: IlustracionesNuevas/Barra de navegaciвn/Creditos/R_BarraNavegecion_Creditos.png]
Output: 3000 w 6501 h, Input: 5001 w 10835 h
[47.16% reduction | 740 kb -> 391 kb: IlustracionesNuevas/Barra de navegaciвn/Introducciвn/R_IR_Migrantes_bienvenida-01-01.jpg]
Output: 12500 w 2500 h, Input: 20834 w 4167 h
[66.9% reduction | 725 kb -> 240 kb: IlustracionesNuevas/Mвdulo 2/2.1 MВxico y EE.UU/IR_2.3-EsquemaSituacionUSA.gif]
[61.26% reduction | 635 kb -> 246 kb: IlustracionesNuevas/Mвdulo 5/Extras/IR_IR_UsodeServicios.gif]
[-8.33% reduction | 600 kb -> 650 kb: IlustracionesNuevas/Mвdulo 4/4B. Servicios pгblicos para familias de bajos recursos/4B1. Medicaid/4B. Requisitos para aplicar a Medicaid/R_u4_requisitosParaAplicar-01.png]
Output: 3375 w 2700 h, Input: 5625 w 4500 h
[-2.67% reduction | 600 kb

In [ ]:
# Hacer resize a todas las imagenes que sobrepasan dimensiones y threshold de peso
# Solo reduce dimensiones a limite máximo (no reduce a menos de eso, como resize_directory)
imagenes_proyecto.resize_limits_directory(directory_path)

# Hacer resize a todas las imagenes que sobrepasan dimensiones
#imagenes_proyecto.resize_limits_directory(directory_path, all=True)


[99.47% reduction | 2455 kb -> 13 kb: ./Barra de navegaciвn/Introducciвn/R_Migrantes_bienvenida-01-01.jpg]
Output: 999 w 200 h, Input: 20834 w 4167 h
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_R_R_R_R_Fin De Curso-01.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_R_R_R_Fin De Curso-01.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_R_R_Fin De Curso-01.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_R_Fin De Curso-01.png.
[71.96% reduction | 1662 kb -> 466 kb: ./Barra de navegaciвn/Fin de curso/R_Fin De Curso-01.png]
Output: 1000 w 2166 h, Input: 5000 w 10834 h
[94.84% reduction | 1609 kb -> 83 kb: ./Barra de navegaciвn/Introducciвn/R_CVA_migrantes_finales_1-01.jpg]
Output: 1000 w 707 h, Input: 3508 w 2481 h
[69.63% reduction | 1472 kb -> 447 kb: ./Barra de navegaciвn/Fin de curso/R_Fin De

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (104183334 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[88.28% reduction | 1032 kb -> 121 kb: ./Barra de navegaciвn/Metodologбa/R_BarraNavegecion_Metodologia.png]
Output: 1000 w 1500 h, Input: 8334 w 12501 h
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Metodologбa/R_R_R_BarraNavegecion_Metodologia.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Metodologбa/R_R_BarraNavegecion_Metodologia.png.


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:2918: DecompressionBombWarning: Image size (100020001 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


[90.2% reduction | 939 kb -> 92 kb: ./Barra de navegaciвn/Polбticas/R_BarraNavegacion_Politicas.png]
Output: 1000 w 1000 h, Input: 10001 w 10001 h
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Polбticas/R_R_R_BarraNavegacion_Politicas.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Polбticas/R_R_BarraNavegacion_Politicas.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Creditos/R_R_R_BarraNavegecion_Creditos.png.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Creditos/R_R_BarraNavegecion_Creditos.png.
[71.16% reduction | 912 kb -> 263 kb: ./Barra de navegaciвn/Creditos/R_BarraNavegecion_Creditos.png]
Output: 1000 w 2166 h, Input: 5001 w 10835 h
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Introducciвn/R_R_R_Migrantes_bienvenida-01-01.jpg.
Info: skipping, file already exists ./imagenes_recortadas/Barra de navegaciвn/Introduc

## **Exportar**

In [206]:
# Guardar solo las versiones más ligeras de cada una de las imagenes generadas
imagenes_proyecto.save_only_smallest_modified_files(directory_path)


In [ ]:
# Mover todas las imagenes a una carpeta
output_directory = "./imagenes_recortadas"
imagenes_proyecto.move_generated(directory_path, output_directory, only_basename=False)

Info: ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_Fin De Curso-01.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Fin de curso/R_Fin De Curso_transparente-01.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Introducciвn/R_Objetivos.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Creditos/R_BarraNavegecion_Creditos.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Introducciвn/IR_Migrantes_bienvenida-01-01.jpg is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Metodologбa/R_BarraNavegecion_Metodologia.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Barra de navegaciвn/Polбticas/R_BarraNavegacion_Politicas.png is contained in ./imagenes_recortadas. Skipping.
Info: ./imagenes_recortadas/Mвdulo

In [ ]:
# Hacer zip para descargar
!zip -r '/content/ilustraciones.zip' /content/IlustracionesNuevas/imagenes_recortadas

# os.chdir('/content/IlustracionesNuevas')
# print(os.listdir())
# print(os.getcwd())

In [ ]:
from google.colab import files
files.download('/content/ilustraciones.zip')

# Código descartado

In [ ]:
def reduce_image_size(input_path, output_path, target_size_kb, reduce_quality=True, reduce_dimensions=True,
                      max_quality_reduction=0.8, max_dimension_reduction=0.8):
    image = Image.open(input_path)

    # Calculate the current size of the image in bytes
    current_size_bytes = os.path.getsize(input_path)

    while current_size_bytes > target_size_kb * 1024:

      if counter >= len(dimensions):
        print(f"Image couldn't be reduced to size {target_size_kb} within the maximum thresholds of: {max_quality_reduction} (quality) and {max_dimension_reduction} (dimension)")
        break;
      counter += 1
      continue
      img_copy = image.copy()

      # Reduce size by diminishing quality
      if reduce_quality:
        img_copy.save(output_path, optimize=True, quality=quality[counter])
        img_copy = Image.open(output_path)

      # Reduce size by reducing dimensions
      if reduce_dimensions:
        new_width = int(image.width * dimensions[counter])
        new_height = int(image.height * dimensions[counter])
        resized_image = img_copy.resize((new_width, new_height), Image.ANTIALIAS)
        resized_image.save(output_path, optimize=True)

      # Calculate the new size of the image
      current_size_bytes = os.path.getsize(output_path)

      counter += 1
      # Break the loop if both reduction methods are disabled or reached the maximum reduction limits
      if not reduce_quality and not reduce_dimensions:
        break


In [ ]:
# Example usage:
input_image_path = "BarraNavegecion_Creditos.png"
output_image_path = "mod_" + input_image_path
target_size_kb = 500
reduce_quality = True
reduce_dimensions = True
max_quality_reduction = 0.9
max_dimension_reduction = 0.8

reduce_image_size(input_image_path, output_image_path, target_size_kb, reduce_quality, reduce_dimensions,
                  max_quality_reduction, max_dimension_reduction)

NameError: ignored

In [ ]:
["y", "x"] == ["y", "x"]

In [154]:
path = "IlustracionesNuevas/Modulo1/Extras/Unidad1_hospPrivados-01-01.png"
path2 = 'Unidad1_hospPrivados-01-01.png'
print(os.path.basename(path))
print(os.path.basename(path2))

Unidad1_hospPrivados-01-01.png
Unidad1_hospPrivados-01-01.png
